# Step 2. Filter signal probes from noise probes

---
> Created 01.31.2019 (in Spyder) by Mike Schmidt to design our approach to filtering probes by comparing Arnatkeviciute's and Richiardi's approaches. Which are likely to be all noise and no signal?

> Updated 11.04.2019 porting from .py to .ipynb while describing differences between filtering approaches in the manuscript.

---

This file will load the dataframe of annotated probes, all mapped to entrez_ids, from the previous step. It will gather all PACall data from slabs labelled CX, excluding brainstem (BS) and cerebellum (CB). All probes with >50% of samples expressing over noise level will be retained. Probes with <50% of samples above noise will be dropped. This will result in two files, filtered_entrez_ids.csv containing 32,011 rows of probes, and filtered_calls.csv containing 2,748 columns of CX samples.

The AHBA data can be downloaded from the Allen Human Brain Atlas at https://human.brain-map.org/static/download or along with the supplemental data from Arnatkeviciute, et al, at https://figshare.com/articles/AHBAdata/6852911. Their paper is at https://doi.org/10.1016/j.neuroimage.2019.01.011.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

In [2]:
""" Explicitly specifying directories is far more robust than relative paths, and it allows for easier context shifting.
    These can be modified where base_dir is where Arnatkeviciute, et als' supplementary date have been extracted.
"""
base_dir = '/home/mike/Dropbox/projects/AHBAProcessing'
ge_dir = '/home/mike/ge_data'

In [3]:
# Read the source data
original_probes = pd.read_csv(os.path.join(ge_dir, "sourcedata/sub-H03511009/expr/Probes.csv"))
entrez_ids = pd.read_csv(os.path.join(ge_dir, "genome/new_annotation_with_entrez_id.csv"))

In [4]:
# Get Call data from all subjects and combine it all.
subs = ['H03511009', 'H03511012', 'H03511015', 'H03511016', 'H03512001', 'H03512002', ]
call_list = []
for sub in subs:
    print("Reading {}'s call data.".format(sub))
    annot = pd.read_csv(os.path.join(ge_dir, "sourcedata/sub-{}/expr/SampleAnnot.csv".format(sub)))
    samples = list(annot['well_id'])
    print(annot.groupby('slab_type')['slab_type'].count())
    calls = pd.read_csv(os.path.join(ge_dir, "sourcedata/sub-{}/expr/PACall.csv".format(sub)), header=None, index_col=0)
    calls.columns = samples
    call_list.append(calls.loc[:, annot[annot['slab_type'] == 'CX']['well_id']])
calls = pd.concat(call_list, axis=1)

call_stats = pd.DataFrame(index=calls.index, columns=['num_called', ],
    data = calls.apply(sum, axis=1),
)
call_stats['pct_called'] = call_stats['num_called'] / len(calls.columns)

Reading H03511009's call data.
slab_type
BS     26
CB     42
CX    295
Name: slab_type, dtype: int64
Reading H03511012's call data.
slab_type
BS     80
CB     48
CX    401
Name: slab_type, dtype: int64
Reading H03511015's call data.
slab_type
BS     79
CB     62
CX    329
Name: slab_type, dtype: int64
Reading H03511016's call data.
slab_type
BS     59
CB     80
CX    362
Name: slab_type, dtype: int64
Reading H03512001's call data.
slab_type
BS    154
CB     53
CX    739
Name: slab_type, dtype: int64
Reading H03512002's call data.
slab_type
BS    188
CB     83
CX    622
Name: slab_type, dtype: int64


In [5]:
""" Filter calls and call_stats by only the probes we can map back to entrez_ids. """

# calls is a binary matrix of 45,871 probes x 2,748 cortical wellids
calls = calls.loc[entrez_ids['probe_id'], :]

# call_stats collapses wellids into a summary, by probe, of 'hits'.
call_stats = call_stats.loc[entrez_ids['probe_id'], :]
call_stats['gene'] = call_stats.index.map(entrez_ids[['probe_id', 'gene']].set_index('probe_id')['gene'].to_dict())

In [6]:
# Build a function to filter probes based on % threshold
def report_on_call_threshold(threshold):
    print("Probes with > {:0.0%} of samples above noise: {:,} ({:0.0%}), <= {:0.0%} {:,} ({:0.0%})".format(
        threshold,
        (call_stats['pct_called'] > threshold).sum(),
        (call_stats['pct_called'] > threshold).sum() / len(call_stats),
        threshold,
        (call_stats['pct_called'] <= threshold).sum(),
        (call_stats['pct_called'] <= threshold).sum() / len(call_stats),
    ))
    print("These {:,} low-call probes cover {:,} genes.".format(
        (call_stats['pct_called'] <= threshold).sum(),
        len(set(call_stats[call_stats['pct_called'] <= threshold]['gene']))
    ))
    print("Removing them leaves {:,} high-call probes covering {:,} genes.".format(
        (call_stats['pct_called'] > threshold).sum(),
        len(set(call_stats[call_stats['pct_called'] > threshold]['gene']))
    ))

def split_on_threshold(x, threshold):
    pct = pd.Series(x.apply(sum, axis=1), index=x.index)
    pct = pct / len(x.columns)
    return {
        'hi': list(pct[pct > threshold].index),
        'lo': list(pct[pct <= threshold].index)
    }

In [7]:
split_on_50 = split_on_threshold(calls, 0.50)
split_on_00 = split_on_threshold(calls, 0.00)

In [8]:
""" Report on results so far. """

print("Starting with {:,} probes,".format(len(calls)))
print("Arnatkeviciute's splitting on 50 results in {} hi and {} lo.".format(len(split_on_50['hi']), len(split_on_50['lo'])))
print("Richiardi's splitting on 00 results in {} hi and {} lo.".format(len(split_on_00['hi']), len(split_on_00['lo'])))

Starting with 45,871 probes,
Arnatkeviciute's splitting on 50 results in 32010 hi and 13861 lo.
Richiardi's splitting on 00 results in 45867 hi and 4 lo.


In [9]:
# Lets just look at genes with multiple probes, like Arnatkeviciute, et al.
pre_counts = entrez_ids.groupby('entrez_id')['entrez_id'].count()
post_counts = entrez_ids[entrez_ids['probe_id'].isin(split_on_50['hi'])].groupby('entrez_id')['entrez_id'].count()
print("There are {:,} multi-probe genes in the set of annotated probes.".format(len(pre_counts[pre_counts > 1])))
print("There are {:,} multi-probe genes after excluding low-call probes.".format(len(post_counts[post_counts > 1])))

print("Writing file with {:,} probe_ids".format(len(entrez_ids[entrez_ids['probe_id'].isin(split_on_50['hi'])])))
entrez_ids[entrez_ids['probe_id'].isin(split_on_50['hi'])].to_csv(
    os.path.join(ge_dir, 'genome/filtered_entrez_ids.csv')
)
print("Writing file with {:,} CX samples".format(len(calls.columns)))
calls.loc[split_on_50['hi'], :].to_csv(
    os.path.join(ge_dir, 'genome/filtered_calls.csv')
)

There are 17,790 multi-probe genes in the set of annotated probes.
There are 11,202 multi-probe genes after excluding low-call probes.
Writing file with 32,010 probe_ids
Writing file with 2,748 CX samples


In [10]:
# Lets find out where Richiardi and Arnatkeviciute probes land vs calls
rich_probes = pd.read_pickle(os.path.join(ge_dir, "cache/richiardi-probes.df"))
rich_probes['pct_called'] = call_stats.reindex(rich_probes.index)['pct_called']
arna_probes = pd.read_pickle(os.path.join(ge_dir, "cache/fornito-probes.df"))
arna_probes['pct_called'] = call_stats.reindex(arna_probes.index)['pct_called']

In [11]:
# Plot them visually
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(7.5, 4))
sns.distplot(rich_probes['pct_called'].dropna(), ax=ax1)
ax1.set_ylabel("Richiardi")
sns.distplot(arna_probes['pct_called'].dropna(), ax=ax2)
ax2.set_ylabel("Arnatkeviciute")
sns.distplot(call_stats['pct_called'].dropna(), ax=ax3)
ax3.set_ylabel("All")
fig.suptitle("Signal calls by probeset")
fig.savefig(os.path.join(base_dir, "calls_by_probeset.png"))
print(fig)
plt.close(fig)

Figure(540x288)


In [12]:
# Get some numbers on how they differ
overlapping_probes = list(set(rich_probes.index).intersection(set(arna_probes.index)))
overlapping_genes = pd.DataFrame({
    'arna': arna_probes.loc[overlapping_probes, 'entrez_id'],
    'rich': rich_probes.loc[overlapping_probes, 'entrez_id'],
    'orig': original_probes.set_index('probe_id').loc[overlapping_probes, 'entrez_id']
}, index=overlapping_probes)

print("Richiardi has {:,} probes in his list that are ineligible for Aurina's".format(
    len(rich_probes[rich_probes['pct_called'] < 0.50].index)
))
print("Richiardi and Arnatkeviciute have {:,} probes in common.".format(
    len(overlapping_probes)
)) 
rich_vs_arna = overlapping_genes['rich'] == overlapping_genes['arna']
rich_vs_orig = overlapping_genes['rich'] == overlapping_genes['orig']
arna_vs_orig = overlapping_genes['arna'] == overlapping_genes['orig']
print("Of those {:,} probes, {:,} agree and {:,} disagree on which gene they match.".format(
    len(overlapping_probes),
    (rich_vs_orig & rich_vs_arna & arna_vs_orig).sum(),
    (~(rich_vs_orig & rich_vs_arna & arna_vs_orig)).sum()
))
print("  With {:,}, Richardi & Arnatkeviciute agree on re-annotation from original.".format(
    (rich_vs_arna & ~rich_vs_orig).sum()
))
print("  With {:,}, Richardi & Arnatkeviciute re-annotated from original differently.".format(
    (~rich_vs_arna & ~rich_vs_orig & ~arna_vs_orig).sum()
))
print("  With {:,}, Richardi re-annotated, but Arnatkeviciute agreed with original.".format(
    (~rich_vs_orig & arna_vs_orig).sum()
))
print("  With {:,}, Arnatkeviciute re-annotated, but Richardi agreed with original.".format(
    (~arna_vs_orig & rich_vs_orig).sum()
))

Richiardi has 6,145 probes in his list that are ineligible for Aurina's
Richiardi and Arnatkeviciute have 2,003 probes in common.
Of those 2,003 probes, 1,928 agree and 75 disagree on which gene they match.
  With 0, Richardi & Arnatkeviciute agree on re-annotation from original.
  With 75, Richardi & Arnatkeviciute re-annotated from original differently.
  With 0, Richardi re-annotated, but Arnatkeviciute agreed with original.
  With 0, Arnatkeviciute re-annotated, but Richardi agreed with original.


In [13]:
len(arna_probes)

15745

In [14]:
len(rich_probes)

16906

This analysis reveals that Richiardi and Arnatkeviciute used different approaches to filtering out no-call probes. Richiardi only excluded 4 probes for never rising above noise-level. Arnatkeviciute excluded 13,861 for being below noise level in half of the locations. After this, only 2,003 probes are used in both studies (out of 15,745 or 16,906). This seems stunning to me, although we should assume that, roughly speaking, multiple probes representing the same gene should have similar expression levels. This process excludes entirely the ability to detect different transcriptions of the same gene.